In [1]:
import json

def get_tickets():
    with open('support_tickets.json', 'r') as file:
        data = json.load(file)
    return data

def get_ticket(index):
    with open('support_tickets.json', 'r') as file:
        data = json.load(file)
    return data['tickets'][index]

def stringify_ticket(ticket):
    return f"Title: {ticket['title']}\nDescription: {ticket['description']}"

ticket = get_ticket(1)
print(stringify_ticket(ticket))


Title: Request to Block Cases from Specific Region
Description: I need to stop receiving cases from the APAC region as I don't have the language capabilities to handle them effectively. Is there a way to update my assignment rules to exclude these cases?


In [41]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
import markdown


# def load_knowledge_base():
#     with open('knowledge_wiki.md', 'r') as file:
#         kb_content = file.read()
    
#     kb_text = markdown.markdown(kb_content)
#     # text_splitter = RecursiveCharacterTextSplitter(
#     #     chunk_size=2000,
#     #     chunk_overlap=50,
#     #     length_function=len,
#     # )
    
#     # chunks = text_splitter.split_text(kb_text)
#     chunks = [chunk.strip() for chunk in kb_text.split('<h3>') if chunk.strip()]

#     print(chunks)
#     embeddings = OllamaEmbeddings(model="llama3.2:1b")
#     vectorstore = FAISS.from_texts(chunks, embeddings)
#     return vectorstore

def load_knowledge_base():
    with open('knowledge_wiki.md', 'r') as file:
        kb_content = file.read()
    
    # Split content on ### headers
    chunks = [chunk.strip() for chunk in kb_content.split('###') if chunk.strip()]
    
    # Convert markdown to HTML, then strip HTML tags to get clean text
    clean_chunks = []
    for chunk in chunks:
        html = markdown.markdown(chunk)
        # Basic HTML tag removal (you might want to use a more robust HTML parser)
        clean_text = html.replace('<strong>', '').replace('</strong>', '') \
                        .replace('<p>', '').replace('</p>', '') \
                        .replace('<h3>', '').replace('</h3>', '') \
                        .replace('<ol>', '').replace('</ol>', '') \
                        .replace('<li>', '').replace('</li>', '')
        clean_chunks.append(clean_text)
    
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_texts(
        clean_chunks,
        embeddings,
        metadatas=[{"chunk_id": i, "source": "knowledge_wiki"} for i in range(len(clean_chunks))]
    )
    return vectorstore


def find_relevant_knowledge(ticket_string, top_k=3):
    vectorstore = load_knowledge_base()
    relevant_chunks = vectorstore.similarity_search(ticket_string, k=top_k)
    return [doc.page_content for doc in relevant_chunks]

In [42]:
load_knowledge_base()

/var/folders/vj/w8n90pdn5nj0vq3c9w7q0hmc0000gn/T/ipykernel_72739/1151345685.py:46: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/Users/sfkunal/anaconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/sfkunal/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of

In [47]:
ticket = get_ticket(2)
ticket_string = stringify_ticket(ticket)
print("Ticket: ", ticket_string)
relevant_knowledge = find_relevant_knowledge(ticket_string)
print("\nRelevant knowledge base sections:")
for i, chunk in enumerate(relevant_knowledge, 1):
    print(f"\nChunk {i}:")
    print(chunk)

Ticket:  Title: Time Zone Update Required
Description: I recently relocated to the EST time zone, but my working hours are still showing in PST. Need assistance updating my profile to reflect the correct time zone so my availability is properly tracked.

Relevant knowledge base sections:

Chunk 1:
Updating Time Zone Settings
To update your working hours and reflect the correct time zone:
1. Go to Profile Settings: Locate the "Time Zone" section.
2. Select the Correct Time Zone: Choose your current time zone from the dropdown.
3. Verify Your Schedule: Ensure that your work hours align with the new time zone.
4. Save Changes: Refresh the system and test by checking availability.

Chunk 2:
Troubleshooting Auto Assignment Issues
If you are not receiving auto assignments despite being marked as "Available," follow these steps:
1. Verify System Status: Check for ongoing outages or maintenance in the assignment system.
2. Check Queue Configuration: Ensure your queue is properly configured and

In [8]:
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage

chat_model = ChatOllama(model="llama3.2:1b")

messages = [
    SystemMessage(content="You are a helpful AI assistant."),
    HumanMessage(content="Tell me a short joke about programming.")
]

response = chat_model.invoke(messages)
print(response.content)

A programmer walked into a library and asked the librarian, "Do you have any books on Pavlov's dogs and Schrödinger's cat?" The librarian replied, "It rings a bell, but I'm not sure if it's here or not."
